In [2]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random
import re
import os
import shutil

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from time import sleep
from datetime import datetime

In [4]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_rows')

## Sacar links

In [18]:
url = f"https://www.autocasion.com/coches-ocasion"
browser = webdriver.Chrome()
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

browser.quit()

print(f"Total de coches: {total_coches}")
print(f"Total de páginas: {total_paginas}")

Total de coches: 105635
Total de páginas: 6214


In [26]:
data = []
total_anuncios = set()
total_fotos = set()
browser = webdriver.Chrome()
    
for i in tqdm.tqdm(range(1, total_paginas + 1)):
    url = f"https://www.autocasion.com/coches-ocasion?page={i}&direction=desc&sort=updated_at"
    browser.get(url)
    sleep(random.uniform(2,5))
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    coches_disponibles = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))
    
    if coches_disponibles == 0:
        break
        
    else:
    
        # Encontrar todos los anuncios en la página
        anuncios_pagina = soup.find_all("article", class_="anuncio")

        for anuncio in anuncios_pagina:
            try:
                link_anuncio = anuncio.find("button", class_="copiar")["rel"]
                if link_anuncio in total_anuncios:
                    continue
                    
                total_anuncios.add(link_anuncio)
                    
                link_foto = anuncio.find("img")["src"]
                total_fotos.add(link_foto)
                response_img = requests.get(link_foto, stream=True)
                response_img.raise_for_status()
                foto_binaria = response_img.content 

                data.append({
                    "link_anuncio": link_anuncio,
                    "foto_binaria": foto_binaria
                })
            except Exception as e:
                print(f"Error procesando un anuncio: {e}")
                continue

browser.quit()

# Mostrar resultados finales
print(f"Anuncios totales únicos: {len(total_anuncios)}")
print(f"Fotos totales únicas: {len(total_fotos)}")

  5%|▌         | 337/6214 [56:26<17:57:20, 11.00s/it]

Error procesando un anuncio: 404 Client Error: Not Found for url: https://images0.autocasion.com/unsafe/500x/ad/05/2656/596b4054942d4cb67835cf226ce827a0b165793a.jpeg


 10%|█         | 625/6214 [1:49:55<16:22:56, 10.55s/it]


Anuncios totales únicos: 7200
Fotos totales únicas: 7183


In [72]:
df_links = pd.DataFrame(data)
df_links.drop(columns=["foto_binaria"]).to_csv("../bin/anuncios_coches.csv", index=False)

In [85]:
n_divisiones = 5

# Dividir manualmente el DataFrame en partes iguales
tamaño_parte = len(df_links) // n_divisiones
dfs_divididos = [df_links.iloc[i * tamaño_parte:(i + 1) * tamaño_parte] for i in range(n_divisiones - 1)]
dfs_divididos.append(df_links.iloc[(n_divisiones - 1) * tamaño_parte:])

# Guardar cada parte como un archivo pickle separado
for i, df_parte in enumerate(dfs_divididos):
    df_parte.to_pickle(f"../bin/imagenes_coches/imagenes_parte_{i + 1}.pkl")
    print(f"Guardado imagenes_parte_{i + 1}.pkl con {len(df_parte)} registros.")

Guardado imagenes_parte_1.pkl con 1439 registros.
Guardado imagenes_parte_2.pkl con 1439 registros.
Guardado imagenes_parte_3.pkl con 1439 registros.
Guardado imagenes_parte_4.pkl con 1439 registros.
Guardado imagenes_parte_5.pkl con 1443 registros.


In [ ]:
max_registros = 1500

n_divisiones = (len(df_links) + max_registros - 1) // max_registros

# Dividimos manualmente el DataFrame en partes con un máximo de 1500 registros
tamaño_parte = max_registros
dfs_divididos = [df_links.iloc[i * tamaño_parte:(i + 1) * tamaño_parte] for i in range(n_divisiones)]

for i, df_parte in enumerate(dfs_divididos):
    df_parte.to_pickle(f"../bin/imagenes_coches/imagenes_parte_{i + 1}.pkl")
    print(f"Guardado imagenes_parte_{i + 1}.pkl con {len(df_parte)} registros.")

## Sacar coches

In [6]:
df_lista_coches = pd.read_csv('../bin/anuncios_coches.csv')

In [45]:
lista_jorge = df_lista_coches.iloc[:15147]
lista_miguel = df_lista_coches.iloc[15147:30294]
lista_carlos = df_lista_coches.iloc[30294:]

In [8]:
# Función para buscar y extraer un valor específico de cada cadena
def buscar_detalle(texto, campo):
    # Usamos expresiones regulares para capturar el campo deseado seguido de su valor
    pattern = rf"{campo}\s*:\s*([^\n]+)"
    match = re.search(pattern, texto)
    return match.group(1).strip() if match else None

In [10]:
coches_df = dict()
browser = webdriver.Chrome()

for coche in tqdm.tqdm(df_lista_coches['link_anuncio']):
    #print(coche)
    
    try: 
        url_coche = coche
        browser.get(url_coche)
        soup = BeautifulSoup(browser.page_source)
        
    except: 
        continue
        
    
    try:
        if 'Vehículo vendido' in soup.find('div', class_= 'img-ficha').find('p').text.replace('X',''):
            continue
            
    except: 
        
        pass
    
    info_coche = {}
            
    try: 
        info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[4].text
        
    except: 
        info_coche['provincia'] = 'no disponible'
        
    info_coche['link_coche'] = coche
    info_coche['fecha_extraccion'] = datetime.now().strftime("%d/%m/%Y")
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
        
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        try: 
            detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
            for detalle in detalles_carroceria:
                detalle = detalle.text.replace('Medidas','').strip()

                if 'Largo' in detalle: 
                    info_coche['largo'] = detalle
                elif 'Ancho' in detalle: 
                    info_coche['ancho'] = detalle
                elif 'Alto' in detalle: 
                    info_coche['alto'] = detalle
                #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
                elif 'Capacidad del maletero' in detalle: 
                    info_coche['capacidad_maletero'] = detalle
                elif 'Carrocería' in detalle: 
                    info_coche['carroceria'] = detalle
                elif 'Número de plazas' in detalle: 
                    info_coche['num_plazas'] = detalle
                elif 'Número de puertas' in detalle: 
                    info_coche['num_puertas'] = detalle
                elif 'Batalla' in detalle: 
                    info_coche['batalla'] = detalle
                #supongo que si hay dos pesos el segundo será la masa max. autorizada
                elif 'Peso' in detalle: 
                    info_coche['peso'] = detalle
                    
        except:   
            info_coche['largo'] = 'no disponible'
            info_coche['ancho'] = 'no disponible'
            info_coche['alto'] = 'no disponible'
            info_coche['capacidad_maletero'] = 'no disponible'
            info_coche['carroceria'] = 'no disponible'
            info_coche['num_plazas'] = 'no disponible'
            info_coche['num_puertas'] = 'no disponible'
            info_coche['batalla'] = 'no disponible'
            info_coche['peso'] = 'no disponible'
                
           
        try:
            detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
            for detalle in detalles_consumos:
                detalle = ' '.join(detalle.text.replace('\n','').split())

                if 'Consumo medio' in detalle: 
                    info_coche['consumo_medio'] = detalle
                elif 'Consumo en carretera' in detalle: 
                    info_coche['consumo_carretera'] = detalle
                elif 'Consumo urbano' in detalle: 
                    info_coche['consumo_urbano'] = detalle
                elif 'CO2' in detalle: 
                    info_coche['CO2'] = detalle
                elif 'Depósito' in detalle: 
                    info_coche['deposito'] = detalle
                    
        except: 
            info_coche['consumo_medio'] = 'no disponible'
            info_coche['consumo_carretera'] = 'no disponible'
            info_coche['consumo_urbano'] = 'no disponible'
            info_coche['CO2'] = 'no disponible'
            info_coche['deposito'] = 'no disponible'        
        
        try:
            detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
            for detalle in detalles_motor:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Combustible' in detalle: 
                    info_coche['combustible'] = detalle
                elif 'Nº de cilindros' in detalle: 
                    info_coche['cilindros'] = detalle
                elif 'Cilindrada' in detalle: 
                    info_coche['cilindrada'] = detalle
                elif 'Sobrealimentación' in detalle: 
                    info_coche['sobrealimentacion'] = detalle
                    
        except: 
            info_coche['combustible'] = 'no disponible'
            info_coche['cilindros'] = 'no disponible'
            info_coche['cilindrada'] = 'no disponible'
            info_coche['sobrealimentacion'] = 'no disponible'

        try:
            detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
            for detalle in detalles_transmision:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Tipo de transmisión' in detalle: 
                    info_coche['tipo_cambio'] = detalle
                elif 'Tracción' in detalle: 
                    info_coche['traccion'] = detalle
                elif 'Número de marchas' in detalle: 
                    info_coche['num_marchas'] = detalle
                    
        except: 
            info_coche['tipo_cambio'] = 'no disponible'
            info_coche['traccion'] = 'no disponible'
            info_coche['num_marchas'] = 'no disponible'

        try:
            detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
            for detalle in detalles_prestaciones:
                detalle = detalle.text

                if 'Potencia (KW)' in detalle: 
                    info_coche['potencia_kw'] = detalle
                elif 'Potencia (CV)' in detalle: 
                    info_coche['potencia_cv'] = detalle
                elif 'Par' in detalle: 
                    info_coche['par'] = detalle
                elif 'Velocidad' in detalle: 
                    info_coche['velocidad_max'] = detalle
                elif 'Aceleración' in detalle: 
                    info_coche['aceleracion'] = detalle
                    
        except: 
            info_coche['potencia_kw'] = 'no disponible'
            info_coche['potencia_cv'] = 'no disponible'
            info_coche['par'] = 'no disponible'
            info_coche['velocidad_max'] = 'no disponible'
            info_coche['aceleracion'] = 'no disponible'
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(3, 6))
    
browser.quit()


100%|██████████| 7199/7199 [12:12:12<00:00,  6.10s/it]  


In [11]:
df = pd.DataFrame.from_dict(coches_df).T

In [12]:
df.to_csv('../bin/data_def.csv', index=False)

In [16]:
df.head()

,provincia,link_coche,fecha_extraccion,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,carroceria,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,combustible,cilindros,cilindrada,sobrealimentacion,tipo_cambio,traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion
0,Madrid,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles VOLKSWAGEN Polo (2019),Matriculado: 10/2019,93000,COCHES AMB,Profesional,C,12 meses,20950,29044,4053 mm Largo,1751 mm Ancho,1461 mm Alto,Capacidad del maletero (l)355/1.125,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.548,Peso/Masa max. autorizado (kg)1.855,Número de puertas5,"Consumo medio 5,80 litros/100 km","Consumo en carretera 4,90 litros/100 km","Consumo urbano 7,70 litros/100 km",CO2 (g/km) 138 g/km,"Depósito de combustible (l) 40,00 litros",Combustible Gasolina,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.984,Sobrealimentación Turbo,Tipo de transmisión Automática secuencial,TracciónTracción delantera,Número de marchas6,Potencia (KW)147,Potencia (CV)200,Par (Nm)320,Velocidad máxima (km/h)237,"Aceleración 0-100 km/h (s)6,70"
1,Madrid,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles OPEL Corsa (2009),Matriculado: 07/2009,236945,AUTOS CAPRICAR,Profesional,B,No,4000,15660,3999 mm Largo,1737 mm Ancho,1488 mm Alto,Capacidad del maletero (l)285/1.100,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.511,Peso/Masa max. autorizado (kg)1.695,Número de puertas5,"Consumo medio 4,60 litros/100 km","Consumo en carretera 3,80 litros/100 km","Consumo urbano 5,90 litros/100 km",CO2 (g/km) 127 g/km,"Depósito de combustible (l) 45,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.248,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)66,Potencia (CV)90,Par (Nm)200,Velocidad máxima (km/h)172,"Aceleración 0-100 km/h (s)12,70"
2,Murcia,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles SKODA Octavia (2017),Matriculado: 08/2017,44300,SKOMOVIL MURCIA,Profesional,ECO,12 meses,15900,25030,4670 mm Largo,1814 mm Ancho,1458 mm Alto,Capacidad del maletero (l)460/1.450,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.680,Peso/Masa max. autorizado (kg)1.878,Número de puertas5,"Consumo medio 5,70 litros/100 km","Consumo en carretera 4,60 litros/100 km","Consumo urbano 7,40 litros/100 km",CO2 (g/km) 101 g/km,"Depósito de combustible (l) 50,00 litros",Combustible Gasolina/gas,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.395,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)81,Potencia (CV)110,Par (Nm)200,Velocidad máxima (km/h)195,"Aceleración 0-100 km/h (s)10,90"
3,Murcia,https://www.autocasion.com/coches-km0/km-0/sko...,25/11/2024,Detalles SKODA Kamiq (2024),Matriculado: 03/2024,10600,SKOMOVIL MURCIA,Profesional,C,12 meses,24900,no disponible,NaN,NaN,NaN,NaN,"4x4, SUV o pickup",None,NaN,None,5 Puertas,NaN,NaN,NaN,NaN,NaN,Gasolina,NaN,None,NaN,Automático,NaN,None,NaN,110,NaN,NaN,NaN
4,Murcia,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles SKODA Octavia (2024),Matriculado: 02/2024,13673,SKOMOVIL MURCIA,Profesional,0 EMISIONES,12 meses,33500,no disponible,NaN,NaN,NaN,NaN,Familiar,None,NaN,None,5 Puertas,NaN,NaN,NaN,NaN,NaN,Eléctrico,NaN,None,NaN,Automático,NaN,None,NaN,204,NaN,NaN,NaN


## Extracción listado marcas

In [18]:
url = 'https://www.autocasion.com/marcas'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

403


In [20]:
browser = webdriver.Chrome()
url = 'https://www.autocasion.com/marcas'
browser.get(url)
soup = BeautifulSoup(browser.page_source)
marcas = soup.find('ul', class_= 'marcas').find_all('span')


listado_marcas = {}
for index,detalle in enumerate (marcas, start = 1):
    detalle = detalle.text.strip()
    
    listado_marcas[index] = detalle
    
browser.quit()

In [22]:
listado_marcas

{1: 'Abarth',
 2: 'Aiways',
 3: 'Alfa romeo',
 4: 'Alpine',
 5: 'Aston martin',
 6: 'Audi',
 7: 'Bentley',
 8: 'Bmw',
 9: 'Byd',
 10: 'Citroen',
 11: 'Cupra',
 12: 'Dacia',
 13: 'Dfsk',
 14: 'Dongfeng',
 15: 'Dr',
 16: 'Ds',
 17: 'Evo',
 18: 'Ferrari',
 19: 'Fiat',
 20: 'Ford',
 21: 'Honda',
 22: 'Hyundai',
 23: 'Ineos',
 24: 'Invicta',
 25: 'Isuzu',
 26: 'Jaecoo',
 27: 'Jaguar',
 28: 'Jeep',
 29: 'Jiayuan',
 30: 'Kgm / ssangyong',
 31: 'Kia',
 32: 'Lamborghini',
 33: 'Lancia',
 34: 'Land-rover',
 35: 'Leapmotor',
 36: 'Levc',
 37: 'Lexus',
 38: 'Livan',
 39: 'Lotus',
 40: 'Lynk & co',
 41: 'Maserati',
 42: 'Maxus',
 43: 'Mazda',
 44: 'Mercedes-benz',
 45: 'Mg',
 46: 'Mhero',
 47: 'Mini',
 48: 'Mitsubishi',
 49: 'Morgan',
 50: 'Nissan',
 51: 'Omoda',
 52: 'Opel',
 53: 'Peugeot',
 54: 'Polestar',
 55: 'Porsche',
 56: 'Renault',
 57: 'Seat',
 58: 'Seres',
 59: 'Skoda',
 60: 'Smart',
 61: 'Subaru',
 62: 'Suzuki',
 63: 'Swm',
 64: 'Tesla',
 65: 'Toyota',
 66: 'Volkswagen',
 67: 'Volvo',
 6

In [24]:
df_marcas = pd.DataFrame(list(listado_marcas.items()), columns=['id_marca', 'nombre_marca'])

In [26]:
df_marcas.to_csv('../bin/listado_marcas.csv', index=False)

## Sacar links beautiful soup

In [33]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

403


In [9]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(1, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    for pagina_url in tqdm.tqdm(lista_paginas):
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("div", class_="contenido-anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')
            
        time.sleep(random.uniform(1,3))
    
        
    return lista_paginas, anuncios_coches

In [11]:
lista_paginas,anuncios_coches = links_coches()

100%|██████████| 6531/6531 [5:49:42<00:00,  3.21s/it]  


In [11]:
headers = {
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
            }
session = requests.Session()

In [ ]:
coches_df = dict()
browser = webdriver.Chrome()

for coche in tqdm.tqdm(lista_jorge.iloc[:, 0]):
    print(coche)
    
    try: 
        url_coche = coche
        browser.get(url_coche)
        soup = BeautifulSoup(browser.page_source)
        response = session.get(url_coche, headers=headers)
        
    except: 
        continue
        
    #Verificamos que la respuesta es exitosa
    if response.status_code != 200:
        print(f"Error al acceder a {url_coche}, Código de estado: {response.status_code}")
        continue
    
    soup = BeautifulSoup(response.content, "html.parser")
    info_coche = {}
    
    try:
        if 'Vehículo vendido' in soup.find('div', class_= 'img-ficha').find('p').text.replace('X',''):
            continue
            
    except: 
        
        pass
            
    try: 
        info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[4].text
        
    except: 
        info_coche['provincia'] = 'no disponible'
        
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
        
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        try: 
            detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
            for detalle in detalles_carroceria:
                detalle = detalle.text.replace('Medidas','').strip()

                if 'Largo' in detalle: 
                    info_coche['largo'] = detalle
                elif 'Ancho' in detalle: 
                    info_coche['ancho'] = detalle
                elif 'Alto' in detalle: 
                    info_coche['alto'] = detalle
                #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
                elif 'Capacidad del maletero' in detalle: 
                    info_coche['capacidad_maletero'] = detalle
                elif 'Carrocería' in detalle: 
                    info_coche['carroceria'] = detalle
                elif 'Número de plazas' in detalle: 
                    info_coche['num_plazas'] = detalle
                elif 'Número de puertas' in detalle: 
                    info_coche['num_puertas'] = detalle
                elif 'Batalla' in detalle: 
                    info_coche['batalla'] = detalle
                #supongo que si hay dos pesos el segundo será la masa max. autorizada
                elif 'Peso' in detalle: 
                    info_coche['peso'] = detalle
                    
        except:   
            info_coche['largo'] = 'no disponible'
            info_coche['ancho'] = 'no disponible'
            info_coche['alto'] = 'no disponible'
            info_coche['capacidad_maletero'] = 'no disponible'
            info_coche['carroceria'] = 'no disponible'
            info_coche['num_plazas'] = 'no disponible'
            info_coche['num_puertas'] = 'no disponible'
            info_coche['batalla'] = 'no disponible'
            info_coche['peso'] = 'no disponible'
                
           
        try:
            detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
            for detalle in detalles_consumos:
                detalle = ' '.join(detalle.text.replace('\n','').split())

                if 'Consumo medio' in detalle: 
                    info_coche['consumo_medio'] = detalle
                elif 'Consumo en carretera' in detalle: 
                    info_coche['consumo_carretera'] = detalle
                elif 'Consumo urbano' in detalle: 
                    info_coche['consumo_urbano'] = detalle
                elif 'CO2' in detalle: 
                    info_coche['CO2'] = detalle
                elif 'Depósito' in detalle: 
                    info_coche['deposito'] = detalle
                    
        except: 
            info_coche['consumo_medio'] = 'no disponible'
            info_coche['consumo_carretera'] = 'no disponible'
            info_coche['consumo_urbano'] = 'no disponible'
            info_coche['CO2'] = 'no disponible'
            info_coche['deposito'] = 'no disponible'        
        
        try:
            detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
            for detalle in detalles_motor:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Combustible' in detalle: 
                    info_coche['combustible'] = detalle
                elif 'Nº de cilindros' in detalle: 
                    info_coche['cilindros'] = detalle
                elif 'Cilindrada' in detalle: 
                    info_coche['cilindrada'] = detalle
                elif 'Sobrealimentación' in detalle: 
                    info_coche['sobrealimentacion'] = detalle
                    
        except: 
            info_coche['combustible'] = 'no disponible'
            info_coche['cilindros'] = 'no disponible'
            info_coche['cilindrada'] = 'no disponible'
            info_coche['sobrealimentacion'] = 'no disponible'

        try:
            detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
            for detalle in detalles_transmision:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Tipo de transmisión' in detalle: 
                    info_coche['tipo_cambio'] = detalle
                elif 'Tracción' in detalle: 
                    info_coche['traccion'] = detalle
                elif 'Número de marchas' in detalle: 
                    info_coche['num_marchas'] = detalle
                    
        except: 
            info_coche['tipo_cambio'] = 'no disponible'
            info_coche['traccion'] = 'no disponible'
            info_coche['num_marchas'] = 'no disponible'

        try:
            detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
            for detalle in detalles_prestaciones:
                detalle = detalle.text

                if 'Potencia (KW)' in detalle: 
                    info_coche['potencia_kw'] = detalle
                elif 'Potencia (CV)' in detalle: 
                    info_coche['potencia_cv'] = detalle
                elif 'Par' in detalle: 
                    info_coche['par'] = detalle
                elif 'Velocidad' in detalle: 
                    info_coche['velocidad_max'] = detalle
                elif 'Aceleración' in detalle: 
                    info_coche['aceleracion'] = detalle
                    
        except: 
            info_coche['potencia_kw'] = 'no disponible'
            info_coche['potencia_cv'] = 'no disponible'
            info_coche['par'] = 'no disponible'
            info_coche['velocidad_max'] = 'no disponible'
            info_coche['aceleracion'] = 'no disponible'
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(3, 7))
